## Latent Outlier Exposure for Anomaly Detection with Contaminated Data

In anomaly detection, our goal is to identify data points that exhibit systematic deviations from the majority of data in an unlabeled dataset. Usually, it is assumed that we have clean training data without anomalies, but in practice, this may not be the case. To overcome this challenge, [Latent OE-AD](https://proceedings.mlr.press/v162/qiu22b.html) propose a strategy for training our anomaly detector when dealing with unlabeled anomalies. This approach is compatible with a wide range of models. The main idea is to jointly infer binary labels (normal vs. anomalous) for each data point while updating the model parameters. Taking inspiration from the concept of outlier exposure, where synthetically created, labeled anomalies are used, we employ a dual loss method. This means using two losses that share parameters, one for normal data and another for anomalous data.


## Loading libraries

The configuration is loaded from the "config_files" directory, and you can find more detailed information there. For this task, we are utilizing the [thyroid dataset](https://odds.cs.stonybrook.edu/thyroid-disease-dataset/), which should be present in the "DATA" folder.

In [10]:
# ruff: noqa
# type: ignore
import argparse
import os
from config.base import Grid, Config
from torch.utils.data import DataLoader

# from evaluation.Experiments import runExperiment
from evaluation.Kvariants_Eval import KVariantEval
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from utils import Patience
from models.Losses import *
from config.Dataset_Class import *
from copy import deepcopy
from utils import read_config_file
from models.TabNets import TabNets
import torch
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np
from utils import compute_pre_recall_f1
from pathlib import Path
import json
import yaml
import pickle
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import random
import warnings

warnings.filterwarnings("ignore")
from loader.LoadData import load_data
from utils import Logger

# choose the configuration file / this file should be in config_files folder
# Consists of all the details related to the training
config_file = "config_thyroid.yml"
config_file = "config_files/" + config_file
# Choose the dataset name / data file loaded from DATA folder
dataset_name = "thyroid"
# Set the 'contamination' rate to control the percentage of anomalies in the dataset.
contamination = float(0.1)
query_num = int(0)

## Loading Data


In [11]:
# Read the configuration file and create a grid of model configurations using the Grid class.
model_configurations = Grid(config_file, dataset_name)

# Select the first model configuration from the grid. This configuration consists of detailed settings for the model.
model_configuration = Config(**model_configurations[0])

# Get the dataset name from the model configuration.
dataset = model_configuration.dataset

# Create the result folder path for saving results and the model. The path is based on the model configuration details.
result_folder = model_configuration.result_folder + model_configuration.exp_name

# Combine the result folder path with the contamination rate and training method to create the experiment path.
exp_path = os.path.join(result_folder, f"{contamination}_{model_configuration.train_method}")

In [12]:
# KVariantEval class controls the variety of experiments to validate the model.
risk_assesser = KVariantEval(dataset, exp_path, model_configurations, contamination, query_num)

In [13]:
import torch
import torch.nn as nn


class TabNeutralAD(nn.Module):
    """
    Tabular Neutral Autoencoder for Anomaly Detection.

    This class implements a Tabular Neutral Autoencoder for anomaly detection. It consists of an encoder network
    and multiple transformation networks (trans) that help disentangle different aspects of the input data.

    Args:
        model (nn.Module): The autoencoder model for disentangling the input data.
        x_dim (int): The dimensionality of the input data.
        config (dict): A dictionary containing configuration settings for the model.

    Attributes:
        enc (nn.Module): The encoder network.
        trans (nn.Module): The list of transformation networks.
        num_trans (int): The number of transformation networks.
        trans_type (str): The type of transformation used, either 'forward' or 'residual'.
        device (str): The device (CPU or GPU) to perform computations on.
        z_dim (int): The dimensionality of the latent space representation.

    """

    def __init__(self, model, x_dim, config):
        super(TabNeutralAD, self).__init__()

        # Extract encoder and transformation networks from the given model.
        self.enc, self.trans = model._make_nets(x_dim, config)

        # Get the number of transformation networks.
        self.num_trans = config["num_trans"]

        # Get the type of transformation used, either 'forward' or 'residual'.
        self.trans_type = config["trans_type"]

        # Get the device (CPU or GPU) to perform computations on.
        self.device = config["device"]

        # Set the dimensionality of the latent space representation.
        try:
            self.z_dim = config["latent_dim"]
        except:
            if 32 <= x_dim <= 300:
                self.z_dim = 32
            elif x_dim < 32:
                self.z_dim = 2 * x_dim
            else:
                self.z_dim = 64

    def forward(self, x):
        """
        Forward pass of the Tabular Neutral Autoencoder.

        Args:
            x (torch.Tensor): The input data.

        Returns:
            torch.Tensor: The latent space representation of the input data.
        """
        # Convert the input data to the appropriate device (CPU or GPU).
        x = x.type(torch.FloatTensor).to(self.device)

        # Initialize a tensor to store the transformed versions of the input data.
        x_T = torch.empty(x.shape[0], self.num_trans, x.shape[-1]).to(x)

        # Apply the transformation networks to the input data.
        for i in range(self.num_trans):
            mask = self.trans[i](x)
            if self.trans_type == "forward":
                x_T[:, i] = mask
            elif self.trans_type == "residual":
                x_T[:, i] = mask + x

        # Concatenate the original input data with the transformed versions.
        x_cat = torch.cat([x.unsqueeze(1), x_T], 1)

        # Encode the concatenated data to obtain the latent space representation.
        zs = self.enc(x_cat.reshape(-1, x.shape[-1]))
        zs = zs.reshape(x.shape[0], self.num_trans + 1, self.z_dim)

        return zs

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DCL(nn.Module):
    """
    Deep Clustering Loss (DCL) Module.

    This class implements the Deep Clustering Loss (DCL) module, which is used for clustering in unsupervised learning.
    The DCL calculates two types of loss: neighbor loss (loss_n) and anti-neighbor loss (loss_a) based on the similarity
    matrix between the input embeddings (z).

    Args:
        temperature (float, optional): The temperature parameter for the DCL. Default is 0.1.

    Attributes:
        temp (float): The temperature parameter for the DCL.

    """

    def __init__(self, temperature=0.1):
        super(DCL, self).__init__()
        self.temp = temperature

    def forward(self, z):
        """
        Forward pass of the Deep Clustering Loss (DCL) module.

        Args:
            z (torch.Tensor): The input embeddings.

        Returns:
            tuple: A tuple containing two losses: neighbor loss (loss_n) and anti-neighbor loss (loss_a).

        """
        # Normalize the input embeddings (z) along the last dimension (L2 normalization).
        z = F.normalize(z, p=2, dim=-1)

        # Split the input embeddings into the original embedding (z_ori) and transformed embeddings (z_trans).
        z_ori = z[:, 0]  # n, z
        z_trans = z[:, 1:]  # n, k-1, z

        # Get the batch size, number of transformations (k), and dimension of embeddings (z_dim).
        batch_size, num_trans, z_dim = z.shape

        # Calculate the similarity matrix between embeddings using exponential of the dot product with temperature.
        sim_matrix = torch.exp(torch.matmul(z, z.permute(0, 2, 1) / self.temp))  # n, k, k

        # Create a mask to remove the similarity of embeddings with themselves.
        mask = (torch.ones_like(sim_matrix).to(z) - torch.eye(num_trans).unsqueeze(0).to(z)).bool()

        # Apply the mask to the similarity matrix to get similarity values between different embeddings.
        sim_matrix = sim_matrix.masked_select(mask).view(batch_size, num_trans, -1)

        # Calculate the sum of similarities for each transformation (trans_matrix) to use in loss_n.
        trans_matrix = sim_matrix[:, 1:].sum(-1)  # n, k-1

        # Calculate the positive similarity between original and transformed embeddings to use in loss_a.
        pos_sim = torch.exp(torch.sum(z_trans * z_ori.unsqueeze(1), -1) / self.temp)  # n, k-1

        # Calculate the scale factor for loss tensor normalization.
        K = num_trans - 1
        scale = 1 / np.abs(np.log(1.0 / K))

        # Calculate neighbor loss (loss_n) and anti-neighbor loss (loss_a).
        loss_tensor = (torch.log(trans_matrix) - torch.log(pos_sim)) * scale
        loss_n = loss_tensor.mean(1)
        loss_a = -torch.log(1 - pos_sim / trans_matrix) * scale
        loss_a = loss_a.mean(1)

        return loss_n, loss_a

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score


class NeutralAD_trainer:
    """
    Trainer for the NeutralAD model.

    This class implements the trainer for the NeutralAD model, which is used for anomaly detection.
    It includes methods for training the model and detecting outliers in the data.

    Args:
        model (nn.Module): The NeutralAD model.
        loss_function (nn.Module): The loss function used during training.
        config (dict): A dictionary containing the configuration parameters.

    Attributes:
        loss_fun (nn.Module): The loss function used during training.
        device (torch.device): The device on which the model and data are located.
        model (nn.Module): The NeutralAD model.
        train_method (str): The training method used for the model.
        max_epochs (int): The maximum number of training epochs.
        warmup (int): The number of warm-up epochs during training.

    """

    def __init__(self, model, loss_function, config):
        self.loss_fun = loss_function
        self.device = torch.device(config["device"])
        self.model = model.to(self.device)
        self.train_method = config["train_method"]
        self.max_epochs = config["training_epochs"]
        self.warmup = 2

    def _train(self, epoch, train_loader, optimizer):
        """
        Perform a single training epoch.

        Args:
            epoch (int): The current epoch number.
            train_loader (DataLoader): The training data loader.
            optimizer (torch.optim): The optimizer used for training.

        Returns:
            float: The average loss for the current epoch.

        """
        self.model.train()
        loss_all = 0

        for data in train_loader:
            samples = data["sample"]
            labels = data["label"]

            z = self.model(samples)
            loss_n, loss_a = self.loss_fun(z)

            if epoch <= self.warmup:
                if self.train_method == "gt":
                    loss = torch.cat([loss_n[labels == 0], loss_a[labels == 1]], 0)
                    loss_mean = loss.mean()
                else:
                    loss = loss_n
                    loss_mean = loss.mean()
            else:
                score = loss_n - loss_a

                if self.train_method == "blind":
                    loss = loss_n
                    loss_mean = loss.mean()
                elif self.train_method == "loe_hard":
                    _, idx_n = torch.topk(
                        score,
                        int(score.shape[0] * (1 - self.contamination)),
                        largest=False,
                        sorted=False,
                    )
                    _, idx_a = torch.topk(
                        score,
                        int(score.shape[0] * self.contamination),
                        largest=True,
                        sorted=False,
                    )
                    loss = torch.cat([loss_n[idx_n], loss_a[idx_a]], 0)
                    loss_mean = loss.mean()
                elif self.train_method == "loe_soft":
                    _, idx_n = torch.topk(
                        score,
                        int(score.shape[0] * (1 - self.contamination)),
                        largest=False,
                        sorted=False,
                    )
                    _, idx_a = torch.topk(
                        score,
                        int(score.shape[0] * self.contamination),
                        largest=True,
                        sorted=False,
                    )
                    loss = torch.cat([loss_n[idx_n], 0.5 * loss_n[idx_a] + 0.5 * loss_a[idx_a]], 0)
                    loss_mean = loss.mean()
                elif self.train_method == "refine":
                    _, idx_n = torch.topk(
                        loss_n,
                        int(loss_n.shape[0] * (1 - self.contamination)),
                        largest=False,
                        sorted=False,
                    )
                    loss = loss_n[idx_n]
                    loss_mean = loss.mean()
                elif self.train_method == "gt":
                    loss = torch.cat([loss_n[labels == 0], loss_a[labels == 1]], 0)
                    loss_mean = loss.mean()

            optimizer.zero_grad()
            loss_mean.backward()
            optimizer.step()

            loss_all += loss.sum()

        return loss_all.item() / len(train_loader.dataset)

    def detect_outliers(self, loader):
        """
        Detect outliers in the data using the trained model.

        Args:
            loader (DataLoader): The data loader for which to detect outliers.

        Returns:
            tuple: A tuple containing the area under the ROC curve (AUC), average precision (AP),
                   F1 score, anomaly scores, inlier loss, and outlier loss.

        """
        model = self.model
        model.eval()

        loss_in = 0
        loss_out = 0
        target_all = []
        score_all = []
        for data in loader:
            with torch.no_grad():
                samples = data["sample"]
                labels = data["label"]

                z = model(samples)
                loss_n, loss_a = self.loss_fun(z)
                score = loss_n
                loss_in += loss_n[labels == 0].sum()
                loss_out += loss_n[labels == 1].sum()
                target_all.append(labels)
                score_all.append(score)

        score_all = torch.cat(score_all).cpu().numpy()
        target_all = np.concatenate(target_all)
        auc = roc_auc_score(target_all, score_all)
        ap = average_precision_score(target_all, score_all)
        f1 = compute_pre_recall_f1(target_all, score_all)
        return (
            auc,
            ap,
            f1,
            score_all,
            loss_in.item() / (target_all == 0).sum(),
            loss_out.item() / (target_all == 1).sum(),
        )

    def train(
        self,
        train_loader,
        contamination,
        query_num=0,
        optimizer=None,
        scheduler=None,
        validation_loader=None,
        test_loader=None,
        early_stopping=None,
        logger=None,
        log_every=2,
    ):
        """
        Train the NeutralAD model.

        Args:
            train_loader (DataLoader): The data loader for training.
            contamination (float): The contamination rate in the data.
            query_num (int, optional): The query number. Default is 0.
            optimizer (torch.optim, optional): The optimizer used for training. Default is None.
            scheduler (torch.optim.lr_scheduler, optional): The learning rate scheduler. Default is None.
            validation_loader (DataLoader, optional): The data loader for validation. Default is None.
            test_loader (DataLoader, optional): The data loader for testing. Default is None.
            early_stopping (object, optional): The early stopping criteria. Default is None.
            logger (Logger, optional): The logger for logging training progress. Default is None.
            log_every (int, optional): The frequency of logging training progress. Default is 2.

        Returns:
            tuple: A tuple containing the validation loss, validation AUC, test AUC, test AP, test F1 score,
                   test anomaly scores.

        """
        self.contamination = contamination
        early_stopper = early_stopping() if early_stopping is not None else None

        val_auc, val_f1 = -1, -1
        test_auc, test_f1, test_score = None, None, None

        for epoch in range(1, self.max_epochs + 1):
            train_loss = self._train(epoch, train_loader, optimizer)

            if scheduler is not None:
                scheduler.step()

            if test_loader is not None:
                (
                    test_auc,
                    test_ap,
                    test_f1,
                    test_score,
                    testin_loss,
                    testout_loss,
                ) = self.detect_outliers(test_loader)

            if validation_loader is not None:
                (
                    val_auc,
                    val_ap,
                    val_f1,
                    _,
                    valin_loss,
                    valout_loss,
                ) = self.detect_outliers(validation_loader)
                if epoch > self.warmup:
                    if early_stopper is not None and early_stopper.stop(
                        epoch,
                        valin_loss,
                        val_auc,
                        testin_loss,
                        test_auc,
                        test_ap,
                        test_f1,
                        test_score,
                        train_loss,
                    ):
                        break

            if epoch % log_every == 0 or epoch == 1:
                msg = f"Epoch: {epoch}, TR loss: {train_loss}, VAL loss: {valin_loss,valout_loss}, VL auc: {val_auc} VL ap: {val_ap} VL f1: {val_f1} "

                if logger is not None:
                    logger.log(msg)
                    print(msg)
                else:
                    print(msg)

        if early_stopper is not None:
            (
                train_loss,
                val_loss,
                val_auc,
                test_loss,
                test_auc,
                test_ap,
                test_f1,
                test_score,
                best_epoch,
            ) = early_stopper.get_best_vl_metrics()
            msg = (
                f"Stopping at epoch {best_epoch}, TR loss: {train_loss}, VAL loss: {val_loss}, VAL auc: {val_auc},"
                f"TS loss: {test_loss}, TS auc: {test_auc} TS ap: {test_ap} TS f1: {test_f1}"
            )
            if logger is not None:
                logger.log(msg)
                print(msg)
            else:
                print(msg)

        return val_loss, val_auc, test_auc, test_ap, test_f1, test_score

In [16]:
class runExperiment:
    """
    Class for running an experiment using a model configuration.

    This class is used to run experiments with a given model configuration. It includes methods for running
    the test phase of the experiment.

    Args:
        model_configuration (dict): A dictionary containing the configuration parameters for the model.
        exp_path (str): The path to save the experiment results.

    Attributes:
        model_config (Config): The model configuration.
        exp_path (str): The path to save the experiment results.

    """

    def __init__(self, model_configuration, exp_path):
        self.model_config = Config.from_dict(model_configuration)
        self.exp_path = exp_path

    def run_test(self, train_data, val_data, test_data, logger, contamination, query_num):
        """
        Run the test phase of the experiment.

        Args:
            train_data (Dataset): The training dataset.
            val_data (Dataset): The validation dataset.
            test_data (Dataset): The test dataset.
            logger (Logger): The logger for logging experiment progress.
            contamination (float): The contamination rate in the data.
            query_num (int): The query number.

        Returns:
            tuple: A tuple containing the validation AUC, test AUC, test average precision (AP), test F1 score,
                   and test anomaly scores.

        """
        optim_class = self.model_config.optimizer
        sched_class = self.model_config.scheduler
        stopper_class = self.model_config.early_stopper
        network = self.model_config.network

        try:
            x_dim = self.model_config["x_dim"]
        except:
            x_dim = train_data.dim_features
        try:
            batch_size = self.model_config["batch_size"]
        except:
            batch_size = int(np.ceil(len(train_data) / 4))

        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=False)

        if len(val_data) == 0:
            val_loader = None
        else:
            val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=False)

        if len(test_data) == 0:
            test_loader = None
        else:
            test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=False)

        model = TabNeutralAD(network(), x_dim, config=self.model_config)
        optimizer = optim_class(
            model.parameters(),
            lr=self.model_config["learning_rate"],
            weight_decay=self.model_config["l2"],
        )

        if sched_class is not None:
            scheduler = sched_class(optimizer)
        else:
            scheduler = None

        trainer = NeutralAD_trainer(
            model,
            loss_function=DCL(self.model_config["loss_temp"]),
            config=self.model_config,
        )

        val_loss, val_auc, test_auc, test_ap, test_f1, test_score = trainer.train(
            train_loader=train_loader,
            contamination=contamination,
            query_num=query_num,
            optimizer=optimizer,
            scheduler=scheduler,
            validation_loader=val_loader,
            test_loader=test_loader,
            early_stopping=stopper_class,
            logger=logger,
        )

        return val_auc, test_auc, test_ap, test_f1, test_score

In [17]:
risk_assesser.risk_assessment(runExperiment)

Normal Cls: 0
Epoch: 1, TR loss: 0.8143219128579295, VAL loss: (0.6871203805779296, 0.7904305153704704), VL auc: 0.8509597028912568 VL ap: 0.3186165491967505 VL f1: 0.3404255319148936 
Epoch: 2, TR loss: 0.6131055834709985, VAL loss: (0.5245263419636181, 0.6860609663293716), VL auc: 0.8813763261717168 VL ap: 0.3367383721304317 VL f1: 0.3191489361702128 
Epoch: 4, TR loss: 0.3427906755250398, VAL loss: (0.3043727988844141, 0.5000987357281624), VL auc: 0.8860967454560179 VL ap: 0.3472220554909719 VL f1: 0.3617021276595745 
Epoch: 6, TR loss: 0.20723840505307758, VAL loss: (0.17582530881996838, 0.37497317537348324), VL auc: 0.9072692143047216 VL ap: 0.39960394200709337 VL f1: 0.43617021276595747 
Epoch: 8, TR loss: 0.14762498255284814, VAL loss: (0.12608419545906402, 0.2950583113000748), VL auc: 0.9399187810211377 VL ap: 0.4512600674605257 VL f1: 0.44680851063829785 
Epoch: 10, TR loss: 0.13444091079868759, VAL loss: (0.11551638463711596, 0.25192666561045546), VL auc: 0.9490993023497971 V

Epoch: 96, TR loss: 0.12273623204616373, VAL loss: (0.10257888138456277, 0.2255839692785385), VL auc: 0.9816620966528988 VL ap: 0.8261484623718134 VL f1: 0.776595744680851 
Epoch: 98, TR loss: 0.1234413404646774, VAL loss: (0.10260520881125952, 0.22714012227159866), VL auc: 0.981826964238196 VL ap: 0.8331569322487256 VL f1: 0.776595744680851 
Epoch: 100, TR loss: 0.12297064827402257, VAL loss: (0.1025914186495293, 0.23071504146494765), VL auc: 0.9826975807851168 VL ap: 0.8372612630744808 VL f1: 0.776595744680851 
Epoch: 102, TR loss: 0.12286692883999863, VAL loss: (0.1025834615104804, 0.23606371372304064), VL auc: 0.9837909131928777 VL ap: 0.8468218817743446 VL f1: 0.7978723404255319 
Epoch: 104, TR loss: 0.1228806863972444, VAL loss: (0.10258436591940252, 0.2397415485787899), VL auc: 0.98489292284197 VL ap: 0.8593665132648863 VL f1: 0.8191489361702128 
Epoch: 106, TR loss: 0.12263014499778206, VAL loss: (0.10239731092178155, 0.23281113644863696), VL auc: 0.9845545104300442 VL ap: 0.85

Epoch: 190, TR loss: 0.12250018761663302, VAL loss: (0.10237310761145238, 0.23237715376184342), VL auc: 0.9847685490495528 VL ap: 0.8788917439843048 VL f1: 0.7978723404255319 
Epoch: 192, TR loss: 0.1225065435782053, VAL loss: (0.10220680423506322, 0.2399122156995408), VL auc: 0.985526361459165 VL ap: 0.8740975750447225 VL f1: 0.8297872340425532 
Epoch: 194, TR loss: 0.12280640903198421, VAL loss: (0.10227024561688587, 0.23904974917148022), VL auc: 0.9838342993995349 VL ap: 0.8690827423264876 VL f1: 0.8191489361702128 
Epoch: 196, TR loss: 0.12264856309559319, VAL loss: (0.10230399085621007, 0.23772363459810297), VL auc: 0.9836520773315749 VL ap: 0.8598610208267399 VL f1: 0.776595744680851 
Epoch: 198, TR loss: 0.12335820508481705, VAL loss: (0.10209389417439844, 0.22857911536034117), VL auc: 0.9830967338863629 VL ap: 0.8828498268720525 VL f1: 0.8297872340425532 
Epoch: 200, TR loss: 0.12227562275577046, VAL loss: (0.10223516450750238, 0.23631925785795171), VL auc: 0.9848611062904216 V

Epoch: 284, TR loss: 0.12225836232428568, VAL loss: (0.10203978728833181, 0.23412805922487948), VL auc: 0.9854511587009591 VL ap: 0.8895409036639086 VL f1: 0.8404255319148938 
Epoch: 286, TR loss: 0.12279854436978019, VAL loss: (0.10188319178752889, 0.2296563412280793), VL auc: 0.9854656207698449 VL ap: 0.8872398323839246 VL f1: 0.8297872340425532 
Epoch: 288, TR loss: 0.12247027501252371, VAL loss: (0.10190210140159818, 0.23139340826805602), VL auc: 0.9835884442284776 VL ap: 0.8844940480327504 VL f1: 0.8297872340425532 
Epoch: 290, TR loss: 0.12265765264345861, VAL loss: (0.1019221311551582, 0.2266729841841028), VL auc: 0.9828335242326427 VL ap: 0.8843417947998169 VL f1: 0.8191489361702128 
Epoch: 292, TR loss: 0.12267560020592419, VAL loss: (0.10204102359043637, 0.23274468361063205), VL auc: 0.9856449504240281 VL ap: 0.8943129671109299 VL f1: 0.8404255319148938 
Epoch: 294, TR loss: 0.12266741438467481, VAL loss: (0.10215317029678155, 0.2436289888747195), VL auc: 0.9908136938437866 V

Epoch: 74, TR loss: 0.12321904612357747, VAL loss: (0.1025795783602459, 0.23127093213669797), VL auc: 0.9835479504355975 VL ap: 0.8836266524263604 VL f1: 0.7978723404255319 
Epoch: 76, TR loss: 0.12325175953331498, VAL loss: (0.10245208989154261, 0.22211040334498627), VL auc: 0.983021531128157 VL ap: 0.871530916128365 VL f1: 0.7872340425531915 
Epoch: 78, TR loss: 0.12332398297444781, VAL loss: (0.10245544201268522, 0.22822410502332321), VL auc: 0.981928198720396 VL ap: 0.8800468280696396 VL f1: 0.7872340425531915 
Epoch: 80, TR loss: 0.1226813511934808, VAL loss: (0.10240681966145833, 0.23024644242956283), VL auc: 0.9834669628498374 VL ap: 0.8895914830501871 VL f1: 0.7978723404255319 
Epoch: 82, TR loss: 0.12307380754692089, VAL loss: (0.10246351531434798, 0.22877342142957321), VL auc: 0.9832240000925572 VL ap: 0.8861224750003667 VL f1: 0.7978723404255319 
Epoch: 84, TR loss: 0.12304671068606982, VAL loss: (0.10235875323131202, 0.2254658151180186), VL auc: 0.983397544919186 VL ap: 0.8

...

Epoch: 218, TR loss: 0.12273107109499282, VAL loss: (0.10201380834947661, 0.22706461967305935), VL auc: 0.9830388856108199 VL ap: 0.8796047190701727 VL f1: 0.7978723404255319 
Epoch: 220, TR loss: 0.12337417341130537, VAL loss: (0.10197072031706685, 0.22221307551607172), VL auc: 0.9817604387213218 VL ap: 0.8743012997906239 VL f1: 0.7978723404255319 
Epoch: 222, TR loss: 0.12256238043920935, VAL loss: (0.10208722312143148, 0.235517440958226), VL auc: 0.983397544919186 VL ap: 0.893468579286129 VL f1: 0.8191489361702128 
Epoch: 224, TR loss: 0.12297227647569445, VAL loss: (0.10229114659004979, 0.23247479377908908), VL auc: 0.9855234690453878 VL ap: 0.8954105874359755 VL f1: 0.8297872340425532 
Epoch: 226, TR loss: 0.12285413475988895, VAL loss: (0.10209956125048855, 0.23023339535327667), VL auc: 0.9842305600870038 VL ap: 0.8905222106576164 VL f1: 0.8297872340425532 
Epoch: 228, TR loss: 0.12265910906238528, VAL loss: (0.10210289677697203, 0.23062801361083984), VL auc: 0.9846673145673528 V

Epoch: 6, TR loss: 0.23402541464470908, VAL loss: (0.20058917013959693, 0.44047168975180767), VL auc: 0.9424236113521455 VL ap: 0.6118676189767478 VL f1: 0.6170212765957447 
Epoch: 8, TR loss: 0.1556217830319995, VAL loss: (0.1251440997224842, 0.2847403262523895), VL auc: 0.9367486955213865 VL ap: 0.5366359662152811 VL f1: 0.5425531914893617 
Epoch: 10, TR loss: 0.13349892891217724, VAL loss: (0.11290744700595177, 0.24489510312993476), VL auc: 0.9431640692790949 VL ap: 0.5825871314469877 VL f1: 0.5638297872340425 
Epoch: 12, TR loss: 0.13012465272997736, VAL loss: (0.10955026449231495, 0.22478377565424493), VL auc: 0.9433376141057235 VL ap: 0.5980564263346371 VL f1: 0.5638297872340425 
Epoch: 14, TR loss: 0.12817585964268233, VAL loss: (0.10773178025909971, 0.21091552490883686), VL auc: 0.9463920030543889 VL ap: 0.6029113267973862 VL f1: 0.5638297872340425 
Epoch: 16, TR loss: 0.12724655980405652, VAL loss: (0.10662475884641365, 0.20438646762929064), VL auc: 0.9485497437321393 VL ap: 0

Epoch: 102, TR loss: 0.12404264729220203, VAL loss: (0.10220460544272958, 0.20158167088285406), VL auc: 0.9788824870130621 VL ap: 0.8328573391630163 VL f1: 0.776595744680851 
Epoch: 104, TR loss: 0.12232748620718994, VAL loss: (0.10217296772511385, 0.21231616811549409), VL auc: 0.9802274594194347 VL ap: 0.8360227731857839 VL f1: 0.7872340425531915 
Epoch: 106, TR loss: 0.12283449924523143, VAL loss: (0.10215083874717493, 0.20729286112683884), VL auc: 0.9799064014901715 VL ap: 0.8165849306458061 VL f1: 0.776595744680851 
Epoch: 108, TR loss: 0.12314996705356324, VAL loss: (0.10222149050838601, 0.21515180709514212), VL auc: 0.9807133849339953 VL ap: 0.8442619993266764 VL f1: 0.7978723404255319 
Epoch: 110, TR loss: 0.12368724373001942, VAL loss: (0.10214182784727178, 0.20468009786402924), VL auc: 0.9767912718521861 VL ap: 0.8280297585297497 VL f1: 0.776595744680851 
Epoch: 112, TR loss: 0.12301868769215767, VAL loss: (0.10218764570110708, 0.20649246459311627), VL auc: 0.9767797021970777 

Epoch: 196, TR loss: 0.12281151023263506, VAL loss: (0.10211488641818753, 0.19964104510368186), VL auc: 0.9797646732150914 VL ap: 0.825205174308745 VL f1: 0.776595744680851 
Epoch: 198, TR loss: 0.12332134648274978, VAL loss: (0.1019838052057844, 0.20271680710163523), VL auc: 0.9786366318420048 VL ap: 0.8443087460497242 VL f1: 0.7978723404255319 
Epoch: 200, TR loss: 0.1221834052968597, VAL loss: (0.10205095549392078, 0.2138205183313248), VL auc: 0.9811327849316811 VL ap: 0.8559854063129014 VL f1: 0.8085106382978723 
Epoch: 202, TR loss: 0.1226794391768386, VAL loss: (0.10194078355201111, 0.2043106809575507), VL auc: 0.9787320814966507 VL ap: 0.8439179628269905 VL f1: 0.7978723404255319 
Epoch: 204, TR loss: 0.12315504584776905, VAL loss: (0.1020234249544377, 0.206927035717254), VL auc: 0.9805571945900293 VL ap: 0.8517656602669912 VL f1: 0.8085106382978723 
Epoch: 206, TR loss: 0.12333024931024, VAL loss: (0.10193319979279244, 0.19240795297825591), VL auc: 0.9795303876991428 VL ap: 0.8

Epoch: 290, TR loss: 0.12272284046085337, VAL loss: (0.1017750609886912, 0.20330506182731467), VL auc: 0.9790328925294736 VL ap: 0.8476544452255212 VL f1: 0.8085106382978723 
Epoch: 292, TR loss: 0.12288541415714482, VAL loss: (0.10196682886950278, 0.21103700678399268), VL auc: 0.9794696470098225 VL ap: 0.844722638860065 VL f1: 0.8191489361702128 
Epoch: 294, TR loss: 0.12320124793741205, VAL loss: (0.10218579539661501, 0.21112245194455412), VL auc: 0.9800018511448174 VL ap: 0.8370995203050917 VL f1: 0.7978723404255319 
Epoch: 296, TR loss: 0.12281177910997537, VAL loss: (0.10207268620004079, 0.2040798714820375), VL auc: 0.9790473545983595 VL ap: 0.8253199201851014 VL f1: 0.776595744680851 
Epoch: 298, TR loss: 0.12292833743699905, VAL loss: (0.10188602947423872, 0.19560270106538813), VL auc: 0.9789027339095022 VL ap: 0.8501121251491585 VL f1: 0.7978723404255319 
Epoch: 300, TR loss: 0.12273328186423535, VAL loss: (0.10194888174566255, 0.20474997987138463), VL auc: 0.9832297849201116 V